In [31]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import OperationalError
import os
from dotenv import load_dotenv
import psycopg2

# Carregar o arquivo Excel em um DataFrame
dados = pd.read_excel('dados/Relatório Consolidado.xlsx', sheet_name="recebimento")

In [32]:
# Carregar variáveis do arquivo .env
load_dotenv()

# Obter as credenciais do banco de dados do .env
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")

In [33]:
dados.head()

,DATA,CATEGORIA,CLIENTE,ITEM_PCG,TOTAL_RECEBIDO
0,2024-07-18,TELEMEDICINA,PagSmart Pagamentos,TELEMEDICINA VAREJO,78.24
1,2024-07-18,OUTROS,PetPremium,COMISSÃO,9.95
2,2024-07-18,PLANOS ODONTOLOGICOS,OdontoPrime,PLANOS ODONTOLOGICOS,1031.28
3,2024-07-17,TELEMEDICINA,Conecta Assessoria,TELEMEDICINA CORPORATIVO,105.00
4,2024-07-17,SEGURO,ProtegeMais Seguros,SEGURO CONDOMINIAL,75.88


In [34]:
dados.tail()

,DATA,CATEGORIA,CLIENTE,ITEM_PCG,TOTAL_RECEBIDO
449,2024-01-04,TELEMEDICINA,PagSmart Pagamentos,TELEMEDICINA VAREJO,186.74
450,2024-01-03,TELEMEDICINA,PagSmart Pagamentos,TELEMEDICINA VAREJO,243.89
451,2024-01-02,TELEMEDICINA,PagSmart Pagamentos,TELEMEDICINA VAREJO,473.42
452,2024-01-02,SEGURO,SimPro Seguros,SEGURO CONDOMINIAL,95.24
453,2024-01-02,SEGURO,TProtege,SEGURO CONDOMINIAL,4.86


In [35]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   DATA            454 non-null    datetime64[ns]
 1   CATEGORIA       454 non-null    object        
 2   CLIENTE         454 non-null    object        
 3   ITEM_PCG        454 non-null    object        
 4   TOTAL_RECEBIDO  454 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 17.9+ KB


In [36]:
# Criar a string de conexão usando as variáveis de ambiente
postgres_str = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
engine = create_engine(postgres_str)

try:
    with engine.connect() as connection:
        # Criar o esquema 'telemedicina' se não existir
        connection.execute("CREATE SCHEMA IF NOT EXISTS telemedicina;")
        print("Esquema 'telemedicina' criado com sucesso!")

        # Nome do esquema e da tabela no banco de dados
        schema_name = 'telemedicina'
        table_name = 'recebimentos'

        # Criar a tabela no banco de dados
        create_table_query = f"""
        CREATE TABLE IF NOT EXISTS {schema_name}.{table_name} (
            DATA DATE,
            CATEGORIA VARCHAR(255),
            CLIENTE VARCHAR(255),
            ITEM_PCG VARCHAR(255),
            TOTAL_RECEBIDO FLOAT,
            STATUS VARCHAR(10)
        );
        """
        connection.execute(create_table_query)
        print(f"Tabela {schema_name}.{table_name} criada com sucesso!")

        # Inserir os dados no banco de dados, substituindo a tabela existente
        dados.to_sql(table_name, engine, schema=schema_name, if_exists='replace', index=False)
        print("Dados substituídos no banco de dados com sucesso!")

except OperationalError as oe:
    print(f"Erro de conexão com o banco de dados: {oe}")
except Exception as e:
    print(f"Erro ao criar a tabela ou inserir dados no banco de dados: {e}")

Esquema 'telemedicina' criado com sucesso!
Tabela telemedicina.recebimentos criada com sucesso!
Dados substituídos no banco de dados com sucesso!
